In [1]:
import os 
import pandas as pd
import numpy as np
pd.options.display.max_rows = 3000

pathTesis = os.getcwd()
pathBases = os.path.join(pathTesis,'bases/DatosPrecia/PreciosConColumnas')
pathYield = os.path.join(pathTesis,'bases/DatosPrecia/yield')
pathMaturity = os.path.join(pathTesis,'bases/DatosPrecia/tablaMaturity')
pathSpline = os.path.join(pathTesis,'bases/DatosPrecia/tablaSpline')
pathibr = os.path.join(pathTesis,'bases/IBR')
pathndata = os.path.join(pathTesis,'bases/ndatos')

In [2]:
#List of Available bonds for each year
resumenMaturity = pd.read_csv('E:/Documentos/Python/TesisJuanFelipe/bases/ResumenMaturityN.csv',sep=";")

In [3]:
resumenMaturity.head()

,Nemotecnico,YtoM,Ano
0,TFIT02150114,1.0,2013
1,TFIT06140514,1.4,2013
2,TFIT10120914,1.7,2013
3,TFIT02121114,1.9,2013
4,TFIT10281015,2.8,2013


## Tasas Medias y Largas por año

In [4]:
medioLargo = []
for nom in os.listdir(pathYield):
    periodo = int(nom.replace('_yield.csv','')[-8:-4]) #year
    columnasIniciales = resumenMaturity[resumenMaturity['Ano']==periodo]['Nemotecnico'].values #bonds from the year
    print(periodo)
    dataSecundaria = pd.read_csv(os.path.join(pathYield,nom),sep='|')[['Fecha']+list(columnasIniciales)]
    dataSecundaria.columns = ['Fecha'] + list(resumenMaturity[resumenMaturity['Ano']==periodo]['YtoM'].values)
    dataSecundaria.index=dataSecundaria.Fecha
    medioLargo.append(dataSecundaria.drop(['Fecha'],axis=1))    

2013
2014
2015
2016
2017
2018


In [5]:
medioLargo[0].info()

<class 'pandas.core.frame.DataFrame'>
Index: 365 entries, 2013-01-01 to 2013-12-31
Data columns (total 14 columns):
1.0     363 non-null float64
1.4     365 non-null float64
1.7     364 non-null float64
1.9     363 non-null float64
2.8     365 non-null float64
2.9     364 non-null float64
3.5     365 non-null float64
5.8     365 non-null float64
5.9     329 non-null float64
7.6     365 non-null float64
9.3     365 non-null float64
11.6    365 non-null float64
13.7    365 non-null float64
15.3    356 non-null float64
dtypes: float64(14)
memory usage: 42.8+ KB


## Tasas Cortas IBR

In [6]:
def mktasas(x):
    try:
        return float(x[0:5])
    except ValueError:
        return np.nan

In [7]:
cortos = []
for nom in os.listdir(pathibr):
    dataCortos = pd.read_csv(os.path.join(pathibr,nom),sep=',')
    dataCortos['fecha'] = pd.to_datetime(dataCortos['fecha'],dayfirst=True)
    dataCortos=dataCortos[(dataCortos['agente']=='IBR')&(dataCortos['tipo_tasa']=='Tasa nominal')]
    dataCortos['tasas']=dataCortos.valor_tasa.apply(mktasas)
    dataCortos=dataCortos[['fecha','tasas']]
    dataCortos.columns=['fecha',nom.replace('.csv','')]
    cortos.append(dataCortos)

In [8]:
ibr_o1=cortos[0].merge(cortos[3],on='fecha')
ibr_o13=ibr_o1.merge(cortos[2],on='fecha')
ibr=ibr_o13.merge(cortos[1],how='left',on='fecha')
ibr.columns=['Fecha','0.004','0.1','0.3','0.5']
ibr.index=ibr.Fecha
ibr=ibr.drop(['Fecha'],axis=1)

In [ ]:
#import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches

In [ ]:
#ibr.index.names = ['Date']
#ibr.plot()
#plt.ylabel('Rates')
#plt.show()

## Datos para cada año

In [9]:
yearid={'2013':0,'2014':1,'2015':2,'2016':3,'2017':4,'2018':5}

In [18]:
year='2015'

In [19]:
term=ibr[year].merge(medioLargo[yearid[year]],how='right',left_index=True,right_index=True)
term=term.fillna(method='pad', axis=0)

In [20]:
term.head()

,0.004,0.1,0.3,0.5,1.5,1.5,3.8,3.9,4.7,5.6,7.3,9.6,11.7,13.3,15.7
Fecha,,,,,,,,,,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,5.14,5.11,6.10,5.90,5.99,6.45,6.92,7.26,7.47,7.49,NaN
2015-01-02,4.360,4.351,4.359,NaN,5.12,5.07,6.01,5.86,5.97,6.35,6.84,7.23,7.44,7.49,NaN
2015-01-03,4.360,4.351,4.359,NaN,5.12,5.07,6.01,5.85,5.97,6.35,6.84,7.23,7.44,7.49,NaN
2015-01-04,4.360,4.351,4.359,NaN,5.12,5.07,6.01,5.85,5.97,6.35,6.84,7.23,7.44,7.49,NaN
2015-01-05,4.362,4.352,4.362,NaN,5.13,5.14,6.06,5.85,5.99,6.42,6.92,7.26,7.35,7.50,NaN


In [21]:
outdata=term.reset_index()
outdata.to_csv(os.path.join(pathndata,str(year)+'data.csv'),sep='|',index=False)